# Slowed and Reverb Music Video Maker

References to project:

Import relevant libraries for this project.

In [ ]:
import

Enter the name of the song you want to convert:

In [1]:
print('Please enter the name of the song you would like to convert:')
song_name = input('> ')

Please enter the name of the song you would like to convert:
> justin bieber baby


Query spotify playlist to choose the song to convert to mp3: